In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score

data = pd.read_csv('/Users/tengyaolong/Desktop/Anaconda stuff/Learning/bank-full-dataset.csv')

data = data.drop(['contact', 'poutcome', 'Unnamed: 0'], axis=1)

data.head()

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,y
0,58,management,married,tertiary,no,2143,yes,no,5,may,261,1,-1,0,no
1,44,technician,single,secondary,no,29,yes,no,5,may,151,1,-1,0,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,76,1,-1,0,no
3,47,blue-collar,married,NaN,no,1506,yes,no,5,may,92,1,-1,0,no
4,33,NaN,single,NaN,no,1,no,no,5,may,198,1,-1,0,no


In [2]:
enc = LabelEncoder()
features_to_convert = ['job', 'marital', 'default', 'housing', 'loan', 'month', 'y']

for i in features_to_convert:
    data[i] = enc.fit_transform(data[i].astype('str'))

    


In [3]:

data['education'] = data['education'].fillna('unknown')
encoder = ['unknown', 'primary', 'secondary', 'tertiary']

for i, word in enumerate(encoder):
    data['education'] = data['education'].astype('str').str.replace(word, str(i))

data['education'] = data['education'].astype('int64')


In [4]:
outliers = {}

for i in range(0, data.shape[1]):
    max = data[data.columns[i]].mean() + 3*data[data.columns[i]].std()
    min = data[data.columns[i]].mean() - 3*data[data.columns[i]].std()
    outliers[data.columns[i]] = [(data[data.columns[i]]<=min).sum() + (data[data.columns[i]]>=max).sum(), data.shape[0]]
print(outliers)

{'age': [381, 45211], 'job': [0, 45211], 'marital': [0, 45211], 'education': [0, 45211], 'default': [815, 45211], 'balance': [745, 45211], 'housing': [0, 45211], 'loan': [0, 45211], 'day': [0, 45211], 'month': [0, 45211], 'duration': [963, 45211], 'campaign': [840, 45211], 'pdays': [1723, 45211], 'previous': [582, 45211], 'y': [0, 45211]}


In [5]:
X = data.drop('y', axis=1)
Y = data['y']

X_new, X_test, Y_new, Y_test = train_test_split(X, Y, test_size=0.2)

test_ratio = X_test.shape[0]/X_new.shape[0]

X_train, X_dev, Y_train, Y_dev = train_test_split(X_new, Y_new, test_size=test_ratio)



In [6]:
model_tree = DecisionTreeClassifier(min_samples_leaf=100, max_depth=70)
model_tree.fit(X_train, Y_train)


X_sets_tree = [X_train, X_dev, X_test]
Y_sets_tree = [Y_train, Y_dev, Y_test]
precision_tree = []


for i in range(0, len(X_sets_tree)):
    Y_pred_tree = model_tree.predict(X_sets_tree[i])
    scores_tree = precision_score(Y_sets_tree[i], Y_pred_tree)
    precision_tree.append(scores_tree)


print(precision_tree)

[0.6465968586387435, 0.6181474480151229, 0.6033834586466166]


In [15]:
model_ANN = MLPClassifier(max_iter=1000, hidden_layer_sizes=[100,100,50,25,25,50], tol=1e-4)
model_ANN.fit(X_train, Y_train)

X_sets_ANN = [X_train, X_dev, X_test]
Y_sets_ANN = [Y_train, Y_dev, Y_test]
precision_ANN = []


for i in range(0, len(X_sets_ANN)):
    Y_pred_ANN = model_ANN.predict(X_sets_ANN[i])
    scores_ANN = precision_score(Y_sets_ANN[i], Y_pred_ANN)
    precision_ANN.append(scores_ANN)

    
    
print(precision_ANN)

[0.6833996023856859, 0.5418569254185692, 0.5392749244712991]


In [16]:
import os
import pickle

path = os.getcwd() + '/final_model.pkl'
file = open(path, 'wb')
pickle.dump(model_tree, file)